In [114]:
#Satya Shah
#Assigment-2
#UTA ID-1002161494

#Fetching Dataset from Kaggle
import kaggle
import pandas as pd
import zipfile
from sentence_transformers import SentenceTransformer
# from elasticsearch import Elasticsearch

kaggle.api.authenticate()
dataset = "harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows"
kaggle.api.dataset_download_files(dataset)

with zipfile.ZipFile(
    "imdb-dataset-of-top-1000-movies-and-tv-shows.zip", "r"
) as zip_ref:
    zip_ref.extractall(".")

movies = pd.read_csv("imdb_top_1000.csv")
movies = movies[["Series_Title","Genre","Director", "Overview"]]


In [115]:
#Concatenate 
movies['find'] = movies['Director'] + ' ' + movies['Overview'] + ' ' + movies['Series_Title']+' '+movies['Genre']
movies.head()

,Series_Title,Genre,Director,Overview,find
0,The Shawshank Redemption,Drama,Frank Darabont,Two imprisoned men bond over a number of years...,Frank Darabont Two imprisoned men bond over a ...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola An organized crime dynast...
2,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,When the menace known as the Joker wreaks havo...,Christopher Nolan When the menace known as the...
3,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,The early life and career of Vito Corleone in ...,Francis Ford Coppola The early life and career...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,A jury holdout attempts to prevent a miscarria...,Sidney Lumet A jury holdout attempts to preven...


In [116]:
#Connecting to elasticsearch Server
from elasticsearch import Elasticsearch
import base64

# Encode the username and password
username = "satya2610"
password = "satya2610"
credentials = f"{username}:{password}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

es = Elasticsearch(
    "http://localhost:9200",
    headers={"Authorization": f"Basic {encoded_credentials}"},

    request_timeout=300
)

es.ping()


True

In [117]:
movies.isna().value_counts()


Series_Title  Genre  Director  Overview  find 
False         False  False     False     False    1000
dtype: int64

In [118]:
#Applying transformer
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


In [119]:
movies["OverviewVector"] = movies["find"].apply(lambda x: model.encode(x))
movies.head()

,Series_Title,Genre,Director,Overview,find,OverviewVector
0,The Shawshank Redemption,Drama,Frank Darabont,Two imprisoned men bond over a number of years...,Frank Darabont Two imprisoned men bond over a ...,"[0.015105659, 0.07558362, 0.0033601949, 0.0132..."
1,The Godfather,"Crime, Drama",Francis Ford Coppola,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola An organized crime dynast...,"[-0.02563735, 0.10421361, 0.008409551, 0.06385..."
2,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,When the menace known as the Joker wreaks havo...,Christopher Nolan When the menace known as the...,"[0.033051766, 0.045696378, -0.010950332, -0.02..."
3,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,The early life and career of Vito Corleone in ...,Francis Ford Coppola The early life and career...,"[-0.04727467, 0.1247385, 0.0004253284, 0.04073..."
4,12 Angry Men,"Crime, Drama",Sidney Lumet,A jury holdout attempts to prevent a miscarria...,Sidney Lumet A jury holdout attempts to preven...,"[0.015930856, 0.011088762, 0.024285793, 0.0243..."


In [120]:
es.ping()


True

In [121]:
# # Update cluster settings to enable shard allocation
# es.cluster.put_settings(body={"transient": {"cluster.routing.allocation.enable": "all"}})


In [122]:
# cluster_health = es.cluster.health()
# print(cluster_health)


In [123]:
#Creating index in the ES database
from indexMapping import indexMapping

es.indices.create(index="satyashah", mappings=indexMapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'satyashah'})

In [124]:
record_list = movies.to_dict("records")


In [125]:
record_list[0]

{'Series_Title': 'The Shawshank Redemption',
 'Genre': 'Drama',
 'Director': 'Frank Darabont',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'find': 'Frank Darabont Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. The Shawshank Redemption Drama',
 'OverviewVector': array([ 1.51056591e-02,  7.55836219e-02,  3.36019485e-03,  1.32937999e-02,
        -3.50598171e-02,  4.95686345e-02,  2.75694709e-02, -2.94112340e-02,
         3.84093449e-02, -2.03052219e-02, -1.40398908e-02,  5.56562319e-02,
         2.14981418e-02, -3.01957410e-02, -1.35939261e-02, -5.04871309e-02,
        -1.89914200e-02,  2.74084136e-02, -1.40913790e-02,  1.26707982e-02,
         5.29568680e-02,  1.39234019e-02,  1.71606112e-02, -5.26667014e-02,
        -1.32068177e-03, -4.08661291e-02,  1.97804719e-02,  2.53821053e-02,
        -1.41185522e-02,  1.39139863e-02,  

In [126]:
for record in record_list:
    try:
        es.index(index="satyashah", document=record)
    except Exception as e:
        print(e)


In [127]:
es.count(index="satyashah")


ObjectApiResponse({'count': 893, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [128]:
#Writing query and fetching results
input_keyword = "HORROR"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "OverviewVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 5,
    "num_candidates" : 1000, 
}

res = es.knn_search(index="satyashah", knn=query , source=["Series_Title","Genre","Director","Overview"])
res["hits"]["hits"]


[{'_index': 'satyashah',
  '_id': '7908to4B7lFPcOZtxIK7',
  '_score': 0.48773336,
  '_source': {'Director': 'William Friedkin',
   'Series_Title': 'The Exorcist',
   'Overview': 'When a 12-year-old girl is possessed by a mysterious entity, her mother seeks the help of two priests to save her.',
   'Genre': 'Horror'}},
 {'_index': 'satyashah',
  '_id': 'Q908to4B7lFPcOZtxoOq',
  '_score': 0.48731494,
  '_source': {'Director': 'Edgar Wright',
   'Series_Title': 'Shaun of the Dead',
   'Overview': "A man's uneventful life is disrupted by the zombie apocalypse.",
   'Genre': 'Comedy, Horror'}},
 {'_index': 'satyashah',
  '_id': 'mN08to4B7lFPcOZtzoTk',
  '_score': 0.48454478,
  '_source': {'Director': 'John Carpenter',
   'Series_Title': 'Halloween',
   'Overview': 'Fifteen years after murdering his sister on Halloween night 1963, Michael Myers escapes from a mental hospital and returns to the small town of Haddonfield, Illinois to kill again.',
   'Genre': 'Horror, Thriller'}},
 {'_index': 